Je kunt de vraag veranderen

Kies dan voor "Clear All Outputs" en "Run All" om het hele RAG pattern te doorlopen
met een nieuwe vraag.

In [1]:
myDataQuestion = "Wanneer is de open dag?"

In [2]:
#inititalize the notebook
import os
import textwrap
import pymongo
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

mongo_user = os.getenv("COSMOS_MONGO_USER")
mongo_pwd = os.getenv("COSMOS_MONGO_PWD")
mongo_server = os.getenv("COSMOS_MONGO_SERVER")
mongo_conn = f"mongodb+srv://{mongo_user}:{mongo_pwd}@{mongo_server}?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"


embeddings_deployment_name = "text-embedding-ada-002"
chat_deployment_name = "gpt-4"

databaseName = "hszuyd"
collectionName = "webpagecollection"

mongo_client = pymongo.MongoClient(mongo_conn)
db = mongo_client[databaseName]
collection = db[collectionName]

print("*** init done! ***")


*** init done! ***


RETRIEVE

In [3]:
def vector_search(vector, num_results=3):
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": vector,
                    "path": "contentVector",
                    "k": num_results 
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results


#create an embedding for the question we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#zoek relevante pagina's op basis van de vraag
results = vector_search(questionEmbedding, 3)

retrievedData = []
print("De top 3 resultaten zijn: ")
for result in results:
    if result["similarityScore"] > 0.8:
        retrievedData.append({"document": result["document"]["filename"],"content": result["document"]["content"]})
    
    print(result["document"]["filename"] + " - " + str(result["similarityScore"]))


De top 3 resultaten zijn: 
hbo-ict-2.md - 0.8554227097206687
hbo-ict-19.md - 0.8442692531897418
hbo-ict-16.md - 0.8275229725269576


AUGMENT

In [7]:
messages = [
    {
        "role": "user",
        "content": myDataQuestion
    },
    {
        "role": "system",
        "content" : f"""
            Je bent een behulpzame assistent
            Je geeft alleen antwoord op basis van de data die je hebt gekregen
            Indien je het antwoord niet kunt vinden, zeg je dat je het antwoord niet weet
            vermeld je bron document bij je antwoord [bron: documentnaam]

            ## data
            {retrievedData}
        """
    }
]


print(json.dumps(messages, indent=4))




[
    {
        "role": "user",
        "content": "Wanneer is de open dag?"
    },
    {
        "role": "system",
        "content": "\n            Je bent een behulpzame assistent\n            Je geeft alleen antwoord op basis van de data die je hebt gekregen\n            Indien je het antwoord niet kunt vinden, zeg je dat je het antwoord niet weet\n            vermeld je bron document bij je antwoord [bron: documentnaam]\n\n            ## data\n            [{'document': 'hbo-ict-2.md', 'content': 'Kom langs op zaterdag 27 januari tussen 10.00 en 15.00u  \\nMeld je aan  \\n* Bezoek een open dag\\n* Download de opleidingsbrochure ICT\\n* Maak je profielwerkstuk bij ons\\n* Houd mij op de hoogte\\n* Mail onze studieadviseur  \\nSchrijf je in voor de opleiding'}, {'document': 'hbo-ict-19.md', 'content': '* Open dagen\\n* Proefstuderen\\n* Hulp bij studiekeuze\\n* Inschrijven'}, {'document': 'hbo-ict-16.md', 'content': 'Wil je ook nog even zelf v\u00f3elen hoe het is om bij ons hbo ICT 

GENERATE

In [8]:
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=messages
    )

lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)





De open dag is op zaterdag 27 januari tussen 10.00 en 15.00u. [bron: hbo-
ict-2.md]
